# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Лабораторная работа 7

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\delia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [8]:
from nltk.tokenize import word_tokenize 
import pandas as pd
recipes = pd.read_csv('recipes_sample.csv').dropna()
words = []
for descr in recipes['description']:
    words += word_tokenize(descr)
new_words = list(set(words))
print(new_words[:10])

['peri-peri', 'designing', '360', 'substitutes', 'ps', 'bush', 'dylan', 'offend', 'animal', 'fast-fry']


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [9]:
import random 
from nltk.metrics import *
our_lst = ['человек', 'открытка', 'змея', 'хозяин', 'Залежалый', 'Агрономия', 'Вконец', 'Сохранить', 'Цемент', 'Шпаты']
lst = [0]*5
for i in range(5):
  lst[i] = random.sample(our_lst,2)
  print(f'Пара: {lst[i]}, расстояние редактирования:{edit_distance(lst[i][0], lst[i][1])}')

Пара: ['человек', 'Агрономия'], расстояние редактирования:8
Пара: ['Шпаты', 'Вконец'], расстояние редактирования:6
Пара: ['Цемент', 'Шпаты'], расстояние редактирования:6
Пара: ['Агрономия', 'открытка'], расстояние редактирования:9
Пара: ['открытка', 'Сохранить'], расстояние редактирования:8


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [10]:
def find_nearest_word(word, words, k):
  dct = {w: edit_distance(w, word) for w in words}
  lst = sorted(dct, key=dct.get)[:k]
  return lst
find_nearest_word('хозяин', our_lst, 3)

['хозяин', 'змея', 'Вконец']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [12]:
import nltk
from nltk.stem import *
import pandas as pd
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\delia\AppData\Roaming\nltk_data...


True

In [18]:
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd

snowball = SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()

dic = {'word': new_words, 'stemmed_word': [snowball.stem(word) for word in new_words], 'normalized_word': [lemmatizer.lemmatize(word) for word in new_words]}
new_df = pd.DataFrame(dic, index=dic['word']).set_index('word')
new_df[:10]

,stemmed_word,normalized_word
word,,
peri-peri,peri-peri,peri-peri
designing,design,designing
360,360,360
substitutes,substitut,substitute
ps,ps,p
bush,bush,bush
dylan,dylan,dylan
offend,offend,offend
animal,anim,animal


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [20]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\delia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [24]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
print(stopwords)
tokens_without_sw = [word for word in words if word.lower() not in stopwords]

top10_before = pd.Series(words).value_counts().head(10)
top10_after = pd.Series(tokens_without_sw).value_counts().head(10) 

stwords_count = len([word for word in words if word.lower() in stopwords])
st_words_ratio = stwords_count/len(words)

print(f'доля стоп-слов от общего количества слов: {st_words_ratio}\n')
print(f'самые частоупотребляемые слова до удаления стоп-слов: {top10_before}\n')
print(f'самые частоупотребляемые слова после удаления стоп-слов: {top10_after}\n')

{'now', 'so', 'were', 'should', 'few', 'most', 'mustn', 'of', 's', 'once', 'our', 'because', 'had', "won't", 'hasn', 'yourselves', "it's", "weren't", 'did', "aren't", 'other', 'at', 'needn', 'doesn', 'by', 'mightn', 'myself', 'ours', "you've", 'a', 'as', 'own', "doesn't", 'but', 'or', 'down', 'not', 'too', 'up', "hadn't", 'itself', 'being', 'couldn', 'very', "shan't", 'in', 'here', 'while', 'again', 'is', 'with', 'further', "don't", 'weren', "hasn't", 'wouldn', 'am', 'd', 'you', 'which', 'does', 'and', 'can', "shouldn't", 'after', 'won', 'how', 'about', 'she', 'more', 'no', 'don', 'whom', 'his', 'we', 'ma', 'against', 'on', 'below', 'each', 'where', 'them', 'are', "should've", "mightn't", 'above', 'll', 'the', 'an', "mustn't", 'do', 'doing', 'same', 'there', 'been', 'who', 'that', 'between', 'my', 'her', 'hers', 'under', 'it', 'has', "didn't", "you'll", 'will', 'have', 'all', 'such', 'be', 'into', 'through', 'this', 'having', 'wasn', 'theirs', 'both', 'herself', 'm', 're', 'shouldn', '

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

df = pd.read_csv('recipes_sample.csv')
random_recipes = df.sample(n=5)

vectorizer = TfidfVectorizer(stop_words='english')
vector = vectorz.fit_transform(random_recipes['description'])

k=0
for i, recipe in random_recipes.iterrows():
    print(f'Рецепт {i+1}: {recipe["name"]}')
    print(f'Векторное описание: {vector[k].toarray()}')
    k+=1

Рецепт 12818: grilled broccolini packets
Векторное описание: [[0.18092934 0.18092934 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.18092934 0.12117049 0.
  0.14597266 0.18092934 0.         0.         0.         0.18092934
  0.18092934 0.36185869 0.18092934 0.18092934 0.18092934 0.18092934
  0.18092934 0.         0.         0.         0.         0.18092934
  0.18092934 0.18092934 0.         0.18092934 0.18092934 0.18092934
  0.18092934 0.         0.         0.         0.18092934 0.18092934
  0.18092934 0.         0.12117049 0.18092934 0.18092934 0.
  0.         0.18092934 0.         0.         0.18092934 0.        ]]
Рецепт 20666: phillips crab cakes
Векторное описание: [[0.         0.         0.28230185 0.28230185 0.         0.
  0.         0.         0.5646037  0.28230185 0.28230185 0.
  0.22775935 0.         0.         0.         0.18906084 0.
  0.         0.         0.28230185 0.         0.       

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [15]:
from scipy.spatial import distance
si = [] 
for i in range(vectors.shape[0]): 
    row = []
    for j in range(vectors.shape[0]):
        sim= 1 - distance.cosine(vectors[i].toarray().ravel(), vectors[j].toarray().ravel())   #ravel()-to change the dimension of vector to 1-D
        row.append(sim) 
    si.append(row) 
s_df = pd.DataFrame(si, columns=random_recs['description']) 
s_df

description,my family loves this chicken recipe. this is my version of outback steakhouse alice springs chicken.,"i have only made these muffins with raspberries, but you could use apples, peaches, blueberries, etc. make sure to use regular sour cream, not low-fat or non-fat as it really does make all the difference in the taste.","this is just like the salad at that "" steakhouse"" , but better at home","scrambling eggs in the microwave is a one dish affair. no muss, no fuss. these come out perfectly and taste delicious. found this in an issue of everyday food. microwave cooking times vary and mine actually needs 55 seconds at each step instead of the 45 seconds listed. don't add extra salt until you've tasted it.","this is a scallop recipe from ina garten's new book in 2008"" back to basics."" it makes a great dish for entertaining so most of the work is done ahead. easy dinner party item if you are serving 6 people no thought needed for the main course. pair with a salad and crusty bread. another winner by her and now you, when you make it. it can also be made using shrimp. enjoy! chefdlh"
0,1.000000,0.000000,0.084089,0.000000,0.033909
1,0.000000,1.000000,0.000000,0.021498,0.045368
2,0.084089,0.000000,1.000000,0.000000,0.049508
3,0.000000,0.021498,0.000000,1.000000,0.018920
4,0.033909,0.045368,0.049508,0.018920,1.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).